In [1]:
import sys
import os

# Add the "src" folder directly to the Python path
sys.path.append(os.path.abspath("src"))


In [2]:
import pandas as pd
from pathlib import Path

# 📂 Set the path manually (because you don't have a src/config.py file with TRANSFORMED_DATA_DIR defined)
TRANSFORMED_DATA_DIR = Path("../data/processed/final_features")

# 📈 Load the merged final Citi Bike dataset
df = pd.read_parquet(TRANSFORMED_DATA_DIR / "rides_citibike_final_2024_with_lags.parquet")

# Display the DataFrame
df


,hour_ts,start_station_name,start_station_id,ride_count,hour,hour_sin,hour_cos,day_of_week,is_holiday_or_weekend,month,...,ride_count_lag_663,ride_count_lag_664,ride_count_lag_665,ride_count_lag_666,ride_count_lag_667,ride_count_lag_668,ride_count_lag_669,ride_count_lag_670,ride_count_lag_671,ride_count_lag_672
0,2024-02-06 11:00:00-05:00,West St & Chambers St,5329,18,11,2.588190e-01,-0.965926,1,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-02-06 11:00:00-05:00,Lafayette St & E 8 St,5788,20,11,2.588190e-01,-0.965926,1,0,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-02-06 12:00:00-05:00,University Pl & E 14 St,5905,25,12,1.224647e-16,-1.000000,1,0,2,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-02-06 12:00:00-05:00,West St & Chambers St,5329,19,12,1.224647e-16,-1.000000,1,0,2,...,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-02-06 12:00:00-05:00,8 Ave & W 31 St,6450,28,12,1.224647e-16,-1.000000,1,0,2,...,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43015,2025-01-31 16:00:00-05:00,Lafayette St & E 8 St,5788,9,16,-8.660254e-01,-0.500000,4,0,1,...,2.0,2.0,5.0,8.0,14.0,4.0,1.0,1.0,1.0,3.0
43016,2025-01-31 16:00:00-05:00,8 Ave & W 31 St,6450,4,16,-8.660254e-01,-0.500000,4,0,1,...,4.0,2.0,2.0,5.0,8.0,14.0,4.0,1.0,1.0,1.0
43017,2025-01-31 16:00:00-05:00,University Pl & E 14 St,5905,9,16,-8.660254e-01,-0.500000,4,0,1,...,9.0,4.0,2.0,2.0,5.0,8.0,14.0,4.0,1.0,1.0
43018,2025-01-31 17:00:00-05:00,Lafayette St & E 8 St,5788,1,17,-9.659258e-01,-0.258819,4,0,1,...,13.0,9.0,4.0,2.0,2.0,5.0,8.0,14.0,4.0,1.0


In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import holidays

from typing import Tuple
from datetime import datetime


In [4]:
# 📦 IMPORTS
import sys
import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pytz import timezone  # ✅ new import for timezone fix

# 🛠 Add parent directory to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

# 📚 Import correct utilities
from src.utils import (
    create_final_features,
    fill_missing_hour_station,
    map_time_of_day,
    to_new_york,
    split_time_series_data
)

# 📂 PATH
features_path = Path(r"C:\Users\MD\Desktop\citi\data\processed\final_features")

# 📥 LOAD DATA
df_2024 = pd.read_parquet(features_path / "rides_citibike_final_2024_with_lags.parquet")


# ✅ Combine
df = pd.concat([df_2024]).reset_index(drop=True)

print(f"✅ Combined dataset shape: {df.shape}")
print(f"✅ Columns: {df.columns.tolist()}")

# 🕒 MAKE CUTOFF TIMEZONE-AWARE
cutoff_date = timezone("America/New_York").localize(datetime(2025, 1, 1, 0, 0, 0))

# ✂️ SPLIT INTO TRAIN/TEST
X_train, y_train, X_test, y_test = split_time_series_data(
    df,
    cutoff_date=cutoff_date,
    target_column="ride_count"  # ✅ your target is ride_count
)

print(f"✅ X_train: {X_train.shape} | y_train: {y_train.shape}")
print(f"✅ X_test: {X_test.shape} | y_test: {y_test.shape}")


✅ Combined dataset shape: (43020, 687)
✅ Columns: ['hour_ts', 'start_station_name', 'start_station_id', 'ride_count', 'hour', 'hour_sin', 'hour_cos', 'day_of_week', 'is_holiday_or_weekend', 'month', 'is_peak_hour', 'day_of_year', 'time_of_day', 'ride_count_roll3', 'target_ride_count', 'ride_count_lag_1', 'ride_count_lag_2', 'ride_count_lag_3', 'ride_count_lag_4', 'ride_count_lag_5', 'ride_count_lag_6', 'ride_count_lag_7', 'ride_count_lag_8', 'ride_count_lag_9', 'ride_count_lag_10', 'ride_count_lag_11', 'ride_count_lag_12', 'ride_count_lag_13', 'ride_count_lag_14', 'ride_count_lag_15', 'ride_count_lag_16', 'ride_count_lag_17', 'ride_count_lag_18', 'ride_count_lag_19', 'ride_count_lag_20', 'ride_count_lag_21', 'ride_count_lag_22', 'ride_count_lag_23', 'ride_count_lag_24', 'ride_count_lag_25', 'ride_count_lag_26', 'ride_count_lag_27', 'ride_count_lag_28', 'ride_count_lag_29', 'ride_count_lag_30', 'ride_count_lag_31', 'ride_count_lag_32', 'ride_count_lag_33', 'ride_count_lag_34', 'ride_cou

In [5]:
# 📦 IMPORTS
import numpy as np

# 🛠 Baseline Model (use previous hour ride count as prediction)
class BaselineModelPreviousHour:
    def __init__(self, lag_column="ride_count_lag_1"):
        self.lag_column = lag_column
    
    def fit(self, X, y):
        # Baseline: No fitting needed
        pass

    def predict(self, X):
        # Predict using the previous hour's ride count
        if self.lag_column not in X.columns:
            raise ValueError(f"Lag column '{self.lag_column}' not found in input data!")
        return X[self.lag_column].values


In [6]:
# 🏋️ Train Baseline Model
baseline_model = BaselineModelPreviousHour(lag_column="ride_count_lag_1")

# Fit (nothing to do, but call for standardization)
baseline_model.fit(X_train, y_train)

# 🧠 Predict
y_pred_baseline = baseline_model.predict(X_test)

# 🧮 Evaluate
from sklearn.metrics import mean_absolute_error, mean_squared_error

mae = mean_absolute_error(y_test, y_pred_baseline)
mse = mean_squared_error(y_test, y_pred_baseline)
rmse = np.sqrt(mse)

print(f"✅ Baseline MAE: {mae:.2f}")
print(f"✅ Baseline RMSE: {rmse:.2f}")


✅ Baseline MAE: 5.56
✅ Baseline RMSE: 8.16


In [7]:
import sys
print(sys.executable)


c:\Users\MD\AppData\Local\Programs\Python\Python312\python.exe


In [8]:
#!pip install pydantic --user


In [9]:
import mlflow
print("MLflow version:", mlflow.__version__)


MLflow version: 2.20.3


In [10]:
import sys
print(sys.executable)

c:\Users\MD\AppData\Local\Programs\Python\Python312\python.exe


In [11]:
#!pip install rich


In [12]:
from dotenv import load_dotenv
load_dotenv()


True

In [13]:
import os
from dotenv import load_dotenv
import mlflow
import dagshub

# Load environment variables
load_dotenv()

# Initialize DagsHub tracking (NO token argument!)
dagshub.init(
    repo_owner="duddukuntamaneesh",
    repo_name="citi",
    mlflow=True
)

print("✅ Connected to DagsHub with MLflow!")
print("Tracking URI:", mlflow.get_tracking_uri())


Accessing as duddukuntamaneesh

Initialized MLflow to track repo "duddukuntamaneesh/citi"

Repository duddukuntamaneesh/citi initialized!

✅ Connected to DagsHub with MLflow!
Tracking URI: https://dagshub.com/duddukuntamaneesh/citi.mlflow


In [14]:
import mlflow
import dagshub
from dotenv import load_dotenv
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

# 🛠 Load environment variables
load_dotenv()

# 🛠 Initialize DagsHub MLflow connection (ONLY ONCE per session)
dagshub.init(repo_owner="duddukuntamaneesh", repo_name="citi", mlflow=True)

# 🏋️ Your training code
baseline_model = BaselineModelPreviousHour(lag_column="ride_count_lag_1")
baseline_model.fit(X_train, y_train)
y_pred_baseline = baseline_model.predict(X_test)

# 📊 Metrics
mae = mean_absolute_error(y_test, y_pred_baseline)
mse = mean_squared_error(y_test, y_pred_baseline)
rmse = np.sqrt(mse)

print(f"✅ Baseline MAE: {mae:.2f}")
print(f"✅ Baseline RMSE: {rmse:.2f}")

# 🧠 Now log both MAE and RMSE to DagsHub / MLflow
with mlflow.start_run(run_name="BaselineModelPreviousHour"):
    mlflow.log_metric("MAE", mae)
    mlflow.log_metric("RMSE", rmse)
    print("✅ MAE and RMSE logged to DagsHub successfully!")


Initialized MLflow to track repo "duddukuntamaneesh/citi"

Repository duddukuntamaneesh/citi initialized!

✅ Baseline MAE: 5.56
✅ Baseline RMSE: 8.16
✅ MAE and RMSE logged to DagsHub successfully!
🏃 View run BaselineModelPreviousHour at: https://dagshub.com/duddukuntamaneesh/citi.mlflow/#/experiments/0/runs/0478cd4d41954d0eb5a187a6519f1a44
🧪 View experiment at: https://dagshub.com/duddukuntamaneesh/citi.mlflow/#/experiments/0
